# Algoritmo KNN
## Por: Matheus Mendonca Lopes, Otávio Santos, Raphael Griffoni, Vinícius Leôncio

Base de dados usada: https://gist.githubusercontent.com/guilhermesilveira/4d1d4a16ccbf6ea4e0a64a38a24ec884/raw/afd05cb0c796d18f3f5a6537053ded308ba94bf7/car-prices.csv

Compreendendo a base de dados: <br>![Tabela](./src/tabela.jpg "Tabela")<br>
- A primeira coluna contém o ID de cada veículo. Essa informação não será necessária para a classificação.
- **milhas_por_ano**: representa quantas milhas em média o carro andou por ano.
- **ano_do_modelo**: representa o ano de fabricação do veículo.
- **preco**: representa o preço do veículo.
- **vendido**: rótulo de classificação para o algoritmo.
<br>
<br>
Para trabalhar com a lista, as variáveis **milhas_por_ano** e **ano_do_modelo** serão utilizados para calcular as milhas rodadas no total, para ser possível realizar o cálculo da distância euclidiana.
<br>
<br>
**Os dados de teste serão os últimos 500 da base de dados do link, os quais serão removidos da base de treino.**

# Definição da Classe do Algoritmo KNN


1. ```__init__(k, x_treino, y_treino)```: 
    - Argumentos:
        * K: Representa quantos vizinhos serão pegos pelo algoritmo;
        * x_treino: lista de treino sem a classificação;
        * y_treino: lista de treino contendo classificações.
    - Funcionamento:<br>
        Apenas recebe variáveis básicas para funcionamento do algoritmo.

2. ```distancia_euclidiana(x_treino_ponto, x_teste_ponto)```:
    - Argumentos:
        * x_treino_ponto: dado contido na lista de treino sem a classificação (Ex: ['0','21801','2000','30941.02','yes']);
        * x_teste_ponto: dado contido na lista de teste sem a classificação (Ex: ['9500','11356','1998','36623.42']).
    - Funcionamento:<br>
        Realiza o cálculo da distância euclidiana, representado pela fórmula:<br>
        ![Dist Euclidiana](src/euclidiana.png "Dist Euclidiana")

3. ```encontra_vizinhos(x_teste_ponto)```:
    - Argumentos:
        * x_teste_ponto: dado contido na lista de teste sem a classificação (Ex: ['9500','11356','1998','36623.42']).
    - Funcionamento:<br>
        Calcula a distância euclidiana do dado de cada um dos dados de treino e as armazena na lista ```distancias``` através de uma list comprehension. 
        Então, se encontra o indíce dos k pontos mais próximos ao ponto de teste, ordenando a lista de distâncias e pegando os indíces dos k primeiros elementos.
        Após obter os k pontos mais próximos, obtêm-se a classificação de cada um deles para classificar o ponto de teste.
        Por fim, determina-se a classificação mais comum, que será a retornada como classificação do ponto de teste.

4. ```previsao(x_teste)```:
    - Argumentos:
        * x_teste: lista com dados que serão testados e classificados.
    - Funcionamento:<br>
        Faz uma list comprehension executando a função ```encontra_vizinhos``` enviando cada dado contido na lista como argumento.
        Após terminar, retorna a lista contendo a classificação como 0 ou 1 (representando no ou yes). O rótulo será convertido para 'no' ou 'yes' posteriormente.




In [130]:
class algoritmo_knn: 
    def __init__(self, k, x_treino, y_treino):
        self.k = k
        self.x_treino = x_treino
        self.y_treino = y_treino
        
    def distancia_euclidiana(self, x_treino_ponto, x_teste_ponto):
        distancia = sum((x_treino_col - x_teste_col) ** 2 for x_treino_col, x_teste_col in zip(x_treino_ponto, x_teste_ponto)) ** 0.5
        return distancia
    
    def encontra_vizinhos(self, x_teste_ponto):
        distancias = [self.distancia_euclidiana(x_treino_ponto, x_teste_ponto) for x_treino_ponto in self.x_treino]
        indices_k_vizinhos = sorted(range(len(distancias)), key = lambda i: distancias[i])[:self.k]
        classificacao_k_vizinhos = [self.y_treino[i] for i in indices_k_vizinhos]
        classificacao_mais_comum = max(set(classificacao_k_vizinhos), key=classificacao_k_vizinhos.count)
        return classificacao_mais_comum

    def previsao(self, x_teste):
        previsoes = [self.encontra_vizinhos(x_teste_ponto) for x_teste_ponto in x_teste]
        return previsoes

# Definição de funções

1. ```calcula_milhas_total(milhas_por_ano, ano)```:
    - Argumentos:
        * milhas_por_ano: representa a quantidade de milhas rodadas pelo carro em média por ano.
        * ano: representa o ano de fabricação do carro.
    - Funcionamento:<br>
        Essa função será utilizada no momento de carga de dados de treino e teste para criar uma lista adaptada, possibilitando melhor análise do algoritmo e o cálculo da distância euclidiana.

2. ```zero_ou_um(linha)```:
    - Argumentos:
        * classificacao: recebe a classificação como **yes** ou **no**, e substitui por 1 e 0, respectivamente.
    - Funcionamento:<br>
        Para melhor trabalho com as classificações, é realizada a substituição da classificação em formato de string para um binário equivalente.

3. ```parametriza_dados(dados)```:
    - Argumentos:
        * dados: recebe a lista de dados completa, com todos os 10000 dados.
    - Funcionamento:<br>
        Formata a lista adaptando cada variável para um formato adequado, pelo fato de chegarem todos em string, e os coloca em uma tupla, para não sofrerem alterações, além de já usar a função ```calcula_milha_total``` para adaptar o dataset. 

4. ```calcula_outliers(dados)```:
    - Argumentos:
        * dados: recebe a lista de dados completa.
    - Funcionamento:<br>
        Recebe a lista de dados completa e descobre os outliers através do cálculo da amplitude de quartis.

5. ```remove_outliers(dados)```:
    - Argumentos:
        * dados: recebe a lista de dados completa.
    - Funcionamento:<br>
        Utiliza ```calcula_outliers``` para descobrir dados inconsistentes numericamente ao dataset e os remove da lista de dados que serão utilizados.

6. ```calcula_acuracia(previsoes, dados_teste)```:
    - Argumentos:
        * previsoes: recebe a lista de previsões de classificações.
        * dados_teste: recebe a lista de dados que foram utilizadas para teste, mas com a classificação original.
    - Funcionamento:<br>
        Compara as previsões obtidas com a classificação original de cada dado, e retorna a porcentagem de precisão.

7. ```junta_previsoes(previsoes, dados_teste)```:
    - Argumentos:
        * previsoes: recebe a lista de previsões de classificações.
        * dados_teste: recebe a lista de dados que foram utilizadas para teste, mas com a classificação original.
    - Funcionamento:<br>
        Forma um novo dataset com header contendo os dados testados e as previsões obtidas. Internamente, faz a troca dos zeros e uns por **no** e **yes**.

In [131]:
def calcula_milhas_total(milhas_por_ano, ano):
    return milhas_por_ano*(2024-ano)

def zero_ou_um(classificacao):
    if classificacao == "yes":
        classificacao = 1
    else:
        classificacao = 0
    return classificacao

def parimetriza_dados(dados):
    dados_parametrizados = [(int(linha[0]),calcula_milhas_total(int(linha[1]), int(linha[2])), float(linha[3]), zero_ou_um(linha[4])) for linha in dados if linha[4] != '']
    return dados_parametrizados

def calcula_outliers(dados):
    dados_ordenados = sorted(dados)
    n = len(dados)
    q1_index = int((n + 1) / 4)
    q3_index = int(3 * ((n + 1) / 4))

    q1 = dados_ordenados[q1_index - 1] 
    q3 = dados_ordenados[q3_index - 1] 

    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    outliers = [x for x in dados if x < limite_inferior or x > limite_superior]
    return outliers

def remove_outliers(dados):
    coluna = [linha[1] for linha in dados]
    outliers = calcula_outliers(coluna)
    dados_sem_outliers = [linha for linha in dados if linha[1] not in outliers]

    return dados_sem_outliers

def calcula_acuracia(previsoes, dados_teste):
    acertos = sum(1 for i in range(len(dados_teste)) if dados_teste[i][-1] == previsoes[i])
    acuracia = (acertos / len(dados_teste)) * 100
    print(f"A precisão desse algorítmo foi de {acuracia:.2f}%")
    
def junta_previsoes(previsoes, dados_teste):
    previsoes_yes_no = ['yes' if prev == 1 else 'no' for prev in previsoes]
    resultados = [[item[0], item[1], item[2], item[3], prev] for item, prev in zip(dados_teste, previsoes_yes_no)]
    resultados.insert(0, ['id', 'milhas_por_ano', 'ano_do_modelo', 'preco', 'previsao_de_venda'])
    return resultados

8. ```prever_vendas(dados)```:
    - Argumentos:
        * dados: dataset completo, sem filtros.
    - Funcionamento:<br>
        Essa é a principal função do programa, pois realiza o tratamento de dados e instância a classe do algoritmo que realizará as operações. Para melhor compreensão, o funcionamento será enumerado por etapa.
        1. O programa embaralha todos os dados para mudar os testes a cada execução.
        2. Os dados são parametrizados para trabalhar com ```milhas_totais``` e, então, os outliers são removidos.
        3. É realizado um slicing dos dados, em que os último 5% dos dados serão utilizados para teste, e suas classificações não serão consideradas.
        4. É criada, então, ```x_treino```, que é a lista adaptada para o algoritmo a fim de realizar o treinamento. Nela, estão contidas as informações ```(milhas_totais, preco)```.
        5. A lista ```y_treino``` se trata apenas das classificações de cada dado de treino, obtidas através da função ```zero_ou_um```.
        6. É criada a lista ```x_teste```, que contém a conversão dos dados de teste para o formato ```(milhas_totais, preco)```.
        7. Após os tratamentos citados acima, é instanciada uma classe do algoritmo, que recebe os argumentos requisitados pelo construtor, como citado no tópico 1 da **Definição da Classe do Algoritmo KNN**. Então, é executada a previsão da classificação dos dados de teste.
        8. Após serem realizadas as previsões, a acurácia do algoritmo é calculada, e o dataset de previsões é montado, unindo as previsões obtidas com os dados de teste.

In [132]:
import random as r

def prever_vendas(dados):
    r.shuffle(dados)
    
    dados_parametrizados = parimetriza_dados(dados)
    dados = remove_outliers(dados_parametrizados)

    indice = int(len(dados) * 0.95)
    dados_classificados = [linha for linha in dados[:indice]]
    dados_nao_classificados = [linha for linha in dados[indice:]]

    x_treino = [(linha[1], linha[2]) for linha in dados_classificados]
    y_treino = [linha[-1] for linha in dados_classificados]   
    x_teste = [(linha[1], linha[2]) for linha in dados_nao_classificados]
    
    knn = algoritmo_knn(k = 50, x_treino = x_treino, y_treino = y_treino)
    
    previsoes = knn.previsao(x_teste)

    calcula_acuracia(previsoes, dados_nao_classificados)
    dataset_previsoes = junta_previsoes(previsoes, dados[indice:])

    return dataset_previsoes

# Execução do código

## Carregando dados de treino:
***Os dados são recebidos através de um arquivo CSV, e o enconding é necessário para remover um bug do Notebook no VSCode em que um caracter é adicionado no início do primeiro dado da lista.**

In [133]:
with open('car-prices.csv', 'r', encoding='utf-8-sig') as ds_carros:
    linhas = ds_carros.readlines()
    dados = [list(linha.strip().split(',')) for linha in linhas]


## Execução do algoritmo:

Envia ```dados[1:]``` para remover a primeira linha do dataset, que contém o cabeçalho da tabela.

In [134]:
venda_prevista = prever_vendas(dados[1:])
venda_prevista